In [2]:
# Imports and boilerplate to make graphs look better
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy
import wave
from IPython.display import Audio

def setup_graph(title='', x_label='', y_label='', fig_size=None):
    fig = plt.figure()
    if fig_size != None:
        fig.set_size_inches(fig_size[0], fig_size[1])
    ax = fig.add_subplot(111)
    ax.set_title(title)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)

# Reinventing neural networks 2

In the previous notebook, we went through experiments that determined that the common neuron representation works pretty well for learning. That representation is characterized by:
* Weighted inputs (which can be adjusted)
* An activation threshold (which can be adjusted)
* Outputs are "all-or-nothing" (in my case, I used `-1` or `1`).

We tested them by generating random configurations of all the "knobs" at our disposal - for each neuron, the input weights and the activation threshold.

Now, we want to adjust the knobs. So, how do we do that? The goal is to adjust the knobs so as to reduce error. We can do that layer by layer, which would be a step in the right direction. But the next question is, what is the "right" direction to step in to reduce error. I remember that the idea of gradient descent is to take the largest step in the direction that will most reduce error, but I don't remember intuitively how it works. Let's see if we can derive it though...

A few thoughts:
* A key lesson I realized from the previous notebook is that we don't want to step in the direction that minimizes the particular training sample at hand ONLY - or else, I would expect that we would hop around quite a bit. Can you learn from samples one-by-one serially? Or do you need to buffer them and consider several as a group?
* Before, we were trying to minimize all the knobs simultaneously. If there are, say, 10 knobs, that is effectively trying to search through a 10-dimensional space (#curse-of-dimensionality). If we, instead, break the knobs up by the layer in the network, that would make the searching much less resource-intensive.
* I wonder if an algorithm like MCMC could be used for searching in these N-dimensional spaces for optimizations?

Let's start by taking another stab at how to represent and run neural networks...

In [12]:
import random
import pprint

def gen_network_weights(neurons_by_layer, init_rand_min=-5, init_rand_max=5):
    neural_network_weights = []
    for layer_index, num_neurons in enumerate(neurons_by_layer):
        if layer_index == 0:
            weights = np.array([1] * num_neurons)
            
        else:
            num_inputs = neurons_by_layer[layer_index - 1]
            weights = np.array([[random.uniform(init_rand_min, init_rand_max) for i in range(num_inputs)]
                                for i in range(num_neurons)])

        neural_network_weights.append(weights)
    return neural_network_weights

In [48]:
pprint.pprint(gen_network_weights([2, 3, 1]))

[array([1, 1]),
 array([[ 4.25283384,  4.81321245],
       [-1.57576796,  0.19632047],
       [ 0.63451474,  4.39674791]]),
 array([[1.52935544, 4.55203107, 3.04998261]])]


In [47]:
pprint.pprint(gen_network_weights([2, 3, 3, 2]))

[array([1, 1]),
 array([[ 2.8655816 ,  4.51039354],
       [ 4.10191434,  2.62328246],
       [-2.95910548, -2.70072649]]),
 array([[ 3.74498396,  3.36186495, -4.44372308],
       [-3.20063982,  1.00369556,  0.73228578],
       [-1.61905121,  2.57913456,  4.26718248]]),
 array([[-2.6834967 , -1.99670531, -0.89801206],
       [ 4.40292339, -3.64891597,  0.12621428]])]


In [60]:
def eval_neuron(layer_index, neuron_index, neuron_input_weights, input_values):
    # Special case for input neurons - just return the input value (identity function)
    if layer_index == 0:
        return input_values[neuron_index]

    # TODO: Stop re-creating these arrays
    if np.sum(np.array(neuron_input_weights) * np.transpose(np.array(input_values))) >= 0:
        return 1
    else:
        return -1

def is_last_layer(layer_list, layer_index):
    return len(layer_list)-1 == layer_index

def run_network(layer_list, sample, sample_label):
    layer_outputs = []
    
    for layer_index, layer in enumerate(layer_list):
        # Special case for output neurons -  return sample label
        if is_last_layer(layer_list, layer_index):
            layer_output = sample_label
        elif layer_index == 0:
            layer_output = sample
        else:
            layer_output = []
            for neuron_index in range(len(layer_list[layer_index])):
                neuron_input_weight = layer_list[layer_index][neuron_index]
                neuron_inputs = layer_list[layer_index - 1]
                print('neuron_weights = {}, input_values = {}'.format(neuron_input_weight, neuron_inputs))
                neuron_eval = eval_neuron(layer_index, neuron_index, neuron_input_weight, neuron_inputs)
                layer_output.append(neuron_eval)

        print('layer_output = {}'.format(layer_output))
        layer_outputs.append(layer_output)
        
    return layer_outputs

#class ResonanceLearner:

In [50]:
test_net = gen_network_weights([2, 3, 1])
run_network(test_net, [1, 1], [-1])

layer_output = [1, 1]
layer_output = [1, 1, -1]
layer_output = [-1]


[[1, 1], [1, 1, -1], [-1]]

In [51]:
test_net = gen_network_weights([2, 3, 1])
run_network(test_net, [5, 10], [1])

layer_output = [5, 10]
layer_output = [1, -1, 1]
layer_output = [1]


[[5, 10], [1, -1, 1], [1]]

In [61]:
test_net = gen_network_weights([2, 3, 3, 2])
run_network(test_net, [5, 10], [1, 2])

layer_output = [5, 10]
neuron_weights = [-3.48080034  0.90779049], input_values = [1 1]
neuron_weights = [ 1.8247945  -2.99530682], input_values = [1 1]
neuron_weights = [ 4.72789622 -3.21629782], input_values = [1 1]
layer_output = [-1, -1, 1]
neuron_weights = [ 1.19385925 -1.68589731  1.10165764], input_values = [[-3.48080034  0.90779049]
 [ 1.8247945  -2.99530682]
 [ 4.72789622 -3.21629782]]
neuron_weights = [ 3.17181281 -2.76863013 -2.73399561], input_values = [[-3.48080034  0.90779049]
 [ 1.8247945  -2.99530682]
 [ 4.72789622 -3.21629782]]
neuron_weights = [-1.6667855  -4.73223135 -0.41407189], input_values = [[-3.48080034  0.90779049]
 [ 1.8247945  -2.99530682]
 [ 4.72789622 -3.21629782]]
layer_output = [1, -1, 1]
layer_output = [1, 2]


[[5, 10], [-1, -1, 1], [1, -1, 1], [1, 2]]